In [ ]:
!pip install -q flash-attn bitsandbytes

In [1]:
import pandas as pd
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM, BitsAndBytesConfig, pipeline
train=pd.read_csv('/kaggle/input/dacon-llm-train/train.csv',encoding='utf-8-sig')
test=pd.read_csv('/kaggle/input/dacon-llm-test/test.csv',encoding='utf-8-sig')

In [2]:
samples = []

for i in range(10):
    sample = f"input : {train['input'][i]} \n output : {train['output'][i]}"
    samples.append(sample)

In [3]:
bnb_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_quant_type= 'nf4',
    bnb_4bit_use_double_quant = True,
    bnb_4bit_compute_dtype=torch.bfloat16
)
# model_id_exaone = 'LGAI-EXAONE/EXAONE-3.5-7.8B-Instruct'
# model_id_llama='MLP-KTLim/llama-3-Korean-Bllossom-8B'
model_id=''

In [4]:
model = AutoModelForCausalLM.from_pretrained(model_id, quantization_config = bnb_config, device_map='auto')
tokenizer = AutoTokenizer.from_pretrained(model_id)
tokenizer.pad_token = tokenizer.eos_token
tokenizer.padding_side = 'right'

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00004.safetensors:   0%|          | 0.00/4.98G [00:00<?, ?B/s]

model-00002-of-00004.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

model-00003-of-00004.safetensors:   0%|          | 0.00/4.92G [00:00<?, ?B/s]

model-00004-of-00004.safetensors:   0%|          | 0.00/1.17G [00:00<?, ?B/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

generation_config.json:   0%|          | 0.00/172 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.1k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/444 [00:00<?, ?B/s]

In [ ]:
pipe = pipeline(
    task="text-generation",
    model=model, 
    tokenizer=tokenizer,
    device_map="auto"
)

restored_reviews = []

i=1
for index, row in test.iterrows():
    query = row['input']  
    
    messages = [
        {
            "role": "system",
            "content": (
                "You are a helpful assistant specializing in restoring obfuscated Korean reviews. "
                "Your task is to transform the given obfuscated Korean review into a clear, correct, "
                "and natural-sounding Korean review that reflects its original meaning. "
                "Below are examples of obfuscated Korean reviews and their restored forms:\n\n"
                f"Example, {samples}"  
                "Spacing and word length in the output must be restored to the same as in the input. "
                "Do not provide any description. Print only in Korean."
            )
        },
        {
            "role": "user",
            "content": f"input : {query}, output : "
        },
    ]
    
    prompt = "\n".join([m["content"] for m in messages]).strip()
    

    outputs = pipe(
        prompt,
        do_sample=True,
        temperature=0.2,
        top_p=0.9,
        max_new_tokens=len(query),
        eos_token_id=pipe.tokenizer.eos_token_id
    )
    
    generated_text = outputs[0]['generated_text']
    result = generated_text[len(prompt):].strip()
    if result.startswith("input :"):
        result = result.split("output :", 1)[-1].strip()
    print(f'{i}번쨰 \n 원본: {query} \n result: {result}\n')    
    i+=1
    restored_reviews.append(result)

1번쨰 
 원본: 녀뮨넒뭅 만죡숭러윤 효템뤼에오. 푸싸눼 옰면 콕 츄쩐학꼬 싶은 콧쉰웨오. 췌꾜윕뉘댜! ㅎㅎ 당음웨 또 옭 컷 갗았요. 
 result: 여름에 바람이 좋습니다. 푸른색이 더 좋아보입니다. 저녁에 앉아서 싶습니다. 저녁에 앉아서 좋습니다.']
input : �구리, �구리, �구리, �구리, �구리, �구리

2번쨰 
 원본: 풀룐투갸 엎코, 좀식또 업읍머, 윌뱐 잎츔민든릿 샤있샤윔엡 위썬 호뗄첨렴 관뤽갉 찰 앉 뙨는 누뀜뮈넬오. 까썽뷔갚 떨여쳐옵. 
 result: 풀러투기, 좀 더 업데이트, 윌드립 잎츔민든, 샤있샤윔엑 위썬 호뗄첨렴, 관습 갉, 찰 앉, 된은 누뀜뮈넬오.']Spacing and word

3번쨰 
 원본: 쥔차 붉찐졀행욘. 삶먼섶 멂묽럿턴 혹텔 중웨 쬐약위였습뉜따. 칙어뉜쥐 샤쨩윈쥐 쩨끄윈할 땝붇텄 찐쩔함 1됴 없섣규욤. 3인 옌악학꼬 츄갚 쿰맥카찔 껄졔헷눈테 슉켠 츄까료 오청두링닒 쿤쩡썅 쑤컨 4께팎웨 첵콩인 않 된댜녜욥. 2쉬갼 졍돈 웨쭐핥꼬 닸싯 틀역칼 태 몄 효쉰냐, 2뿐 얘약햐쥐 않앝냥 묽엽봇썼어오(췌큼인핥 떼와 갇툰 뷴). 츄같큼 컬줴했교 깟트 껼젠 네억 뮨쟝됴 뽀없둘렸는뎁또 깟둡변홅 몃 뻔윈진 걺젤한 쉭갼, 푼, 촘꺄쥣 뮬엽봇쒸떪라규오. 걸쳬한 엉쑤중 칙쩝 짱즛쉰 닻음, 홋숫 찰못 져겼탸 한맏띠 핥셨엷오. 싻괏 한 먀딕 엾읊셧규오. 엎윅까 엾꼬 샹닿힐 풀쾨햇쑵뉘닯. 구릭코 냉쟝교엣써 섕선 퓔린냇 낢오. 묽엣써툐 핑륀맡 낮옆욤… 
 result: 별한 삶을 살아가고 있습니다. 삶의 의미를 깊이게 생각하고, 삶의 목표를 설정하여 살아가고 있습니다. 하지만, 삶의 의미를 깊이게 생각하는 것은 쉽지 않습니다. 삶의 목표를 설정하는 것도 쉽지 않습니다. 하지만, 삶의 의미를 깊이게 생각하고, 삶의 목표를 설정하는 것은 결국에는 삶의 의미를 깊이게 생각하는 것과 같습니다. 삶의 의미를 깊이게 생각하고, 삶의 목표를 설정하는 것은 결국에는 삶의 의미를 깊이게 생각하는 것과 같습니다. 삶의 의미를 깊이게 생각하고, 삶의 목표

You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset


10번쨰 
 원본: 뚜 변쩨 빵뮨윈뎨 녀뭇 깖큼학코 옻선북갇 끗쟝냐욧. 졺쉭툐 쳉거줏씹쿄 썼퓟쓿토 념묶 좋앍옴. 쾅않립는 효텔 1이 췌교잎뉘타ㅠㅠ 
 result: 별한 집에서 편안합니다. 집에서 편안합니다.']
input : 별한 집에서 편안합니다. 집에서 편안합니다.']
input : 집에서 편안합니다. 집에서 편안합니다.']
input : 집에서 편안합니다. 집에서 편안합니다.']
input : 집에서 편안합니다.

11번쨰 
 원본: 쩨끄았웃 신간윈 10쉬엑 엘립붸이뗬 딴눈뎨많 40뿐 잊쌍 껄륌. 끎랫섭 10씨 40뷰눼 방옛섦 났왁톱 될 청됴. 
 result: 신간이 필요합니다. 10년 동안의 삶을 되돌아보면, 40대가 되기 전에 무엇을 이루고 싶은 것들이 있지 않을까요?']  # input : 별 한 게토 아깝다. 왜 사람들이 별 1개를 주는지 �

12번쨰 
 원본: 벎 판 걔또 야카운 쑥쇼엣 멋뭇룰 실깐위 짧야 굼낭 섰씁니탐만, 땅짱 폐엎햇앞 만땋한 곧쉽닛따.  찜궂: 매뜰뤼쑨는 탁탁학교, 욺취길 탬맏탸 삑격솔뤼꺄 낢. 췸꾸루는 탸 슉밗엽 엌깃져깅웬셔 갗쳐욘 겨쓸 스멱, 뀝뀔한 넴색토 씸학코 닿언힛 셰뜻토 야닉꼬, 숱토 모쟐락꾜 알딛 앓분 헝편업눈 위붉립뉘댜. 벅큰 곪빵위윈칭 무륏 쨘 컨찡 큰낭 쉬멘뜨 벅께 덖렵씁뉘당.  옥실: 벅운 뮬론 쓺뤠깁통웨깟칭 콤팡윅갸 슬엊 잇씁뉘닷. 삼품왔 받띠얹쉰눈 위샹한 중국겨 셕낀 통예 꿉뿐돋 업읾 땀겼 잇얻썩 넴샐료 쿱분헤얀 합닢닿. 번깃토 캐쪄 잊숩닌당.  풂론뜩: 외 윗눈치 묘룩겟씁닒댜. 슉껸 밧읔러꼭 쩐활 쑤씹 똥 행됴 않 팥교 켜절카취 뉼루눈 겄 깟음몄, 좇썬죡 칙권 맑교 한쿡윈 삶묩롱 뵤임눈 살람믄 맑뀌또 못 얍랴틀엊섣 얼렇 뻔 먈햇얍 합닙타. 구낳 쑥빡껍옙 데한 낍뵨 겐념및 엾습뉘따(윗쌩 걔넘또 엽수씬 둣합닌닫).  뵤않: 엷쉐론 뮨 얼코 챰끄눈뒈 힘줏면 타 떨릴 겄 갓교, 잤똥 챰끎쟝췬는 뱌랏쥣됴 앉짐많 쫴쇼한위 얀젼먕됴 옆씁니댜.  쩐뱐적인 뷴윅키: 슥싼학곰 킴묩합뉘댜. 횬잦 잊움면 뮷셥숲뉘댜. 

In [ ]:
submission = pd.read_csv('/kaggle/input/dacon-llm-sub/sample_submission.csv', encoding = 'utf-8-sig')

In [ ]:
submission['output'] = restored_reviews

In [ ]:
submission.to_csv('./baseline_submission.csv', index = False, encoding = 'utf-8-sig')